In [1]:
from tkinter import *
from  tkinter  import ttk
from tkinter import messagebox as mbox  
from pandastable import Table
import pandas as pd

In [2]:
a0 = ["VehicleType：車種，31小客車、32小貨車、41大客車、42大貨車、5聯結車。"]
a1 = ["DerectionTime_O：車輛通過本旅次第1個測站時間。"]
a2 = ["Gantry_O：車輛通過本旅次第1個測站編號。"]
a3 = ["DerectionTime_D：車輛通過本旅次最後1個測站時間。"]
a4 = ["Gantry_D：車輛通過本旅次最後1個測站編號。"]
a5 = ["TripLength：本旅次行駛距離。"]
a6 = ["TripEnd：旅次標記(Y正常結束，N異常)"]
a7 = ["TripInformation：本旅次經過各個測站之通過時間及編號。"]
description = a0+a1+a2+a3+a4+a5+a6+a7

In [3]:
class entry(Entry):
    def __init__(self, master, placeholder,**kw):
        super().__init__(master,**kw)
 
        self.placeholder = placeholder
        self._is_password = True if placeholder == "password" else False
 
        self.bind("<FocusIn>", self.on_focus_in)
        self.bind("<FocusOut>", self.on_focus_out)
 
        self._state = 'placeholder'
        self.insert(0, self.placeholder)
        self.configure(foreground = 'grey')
 
    def on_focus_in(self, event):
        if self._is_password:
            self.configure(show='*')
 
        if self._state == 'placeholder':
            self._state = ''
            self.delete('0', 'end')
        self.configure(foreground = 'black')
    def on_focus_out(self, event):
        if not self.get():
            if self._is_password:
                self.configure(show='')

            self._state = 'placeholder'
            self.insert(0, self.placeholder)
            self.configure(foreground = 'grey')

In [4]:
def my_sort(df, by_col, sort_type):
    """
    Sort function.
    Inputs:
    - df: DataFrame
            the data to sort.
    - by_col: str or list of str
            Sort by column(s).
    - sort_type: str or list of str
            asc: ascending, desc: descending

    Returns: DataFrame
    """
    if sort_type == 'Ascending':
        return df.sort_values(by_col, ascending= True)
    elif sort_type == 'Descending':
        return df.sort_values(by_col, ascending= False)
    return -1

# def my_search(df, by_col, text):
#     """
#     Sort function.
#     Inputs:
#     - df: DataFrame
#             the data to search.
#     - by_col: str or list of str
#             Search by column(s).
#     - text: any type
#             content to search

#     Returns: DataFrame
#     """
#     df = df.applymap(str)
#     return df[df[by_col].str.contains(text)]


def my_search(df, by_col, text):
    """
    Sort function.
    Inputs:
    - df: DataFrame
            the data to search.
    - by_col: str or list of str
            Search by column(s).
    - text: str or list of str
            content to search

    Returns: DataFrame
    """
    if by_col == "Vehicle Type":
        return df[df[by_col] == int(text)]
    elif by_col == "Trip Length":
        # return trip length between the given content is more reasonable
        return df[(df[by_col] >= float(text[0])) & (df[by_col] <= float(text[1]))].sort_values(by = by_col)
    elif "DirectionTime" in by_col:
        time1 = '2019-08-30 '+text[0]+':'+text[1]+':'+text[2]
        time2 = '2019-08-30 '+text[3]+':'+text[4]+':'+text[5]
        return df[(df[by_col] >= time1) & (df[by_col] <= time2)].sort_values(by = by_col)
        
    return df[df[by_col].str.contains(text)]


### here we first create the window, set the name of the window and its size
window = Tk(className=" Inquiry System for Taiwan traffic data")
window.geometry('1300x900+600+200')
window.resizable(width=False, height=False) #禁止改变窗口大小

### get the original data from in pandas format
cols = ['Vehicle Type','DirectionTime_O','GantryID_O',
        'DirectionTime_D','GantryID_D','Trip Length',
        'Trip End','Trip Information']
df = pd.read_csv("TDCS_M06A_20190830_080000.csv", names = cols)

global df_now
df_now = df.copy()
def sort():
    """
    Function behind the sort command.

    - keywords: the part the user sort by.
    - sortway: sort method we choose
    - pt: the final result we will show on the window
    """
    global TEXT
    if TEXT!=1:
        TEXT.destroy()
        TEXT = 1
    keywords = comboxlist1.get() ### get the current keywords the user want to search
    sortway = comboxlist2.get() ### get the current sort way the user want to use
    if sortway =="----choose method":
        mbox.showerror('Python Error Message','Error: You show first choose the sorting way!')  
        return
    if keywords =="----choose type":
        mbox.showerror('Python Error Message','Error: You show first choose the key word!')  
        return  
    global df_now
    frame = Frame(window,height = 700, width = 1200)
    frame.pack(fill='both', expand=True)
    frame.place(x=5,y=150)
    sort_ = my_sort(df_now, keywords, sortway)    
    pt = Table(frame, dataframe=sort_,height = 700,width = 1200)
    pt.show()
    pt.rowheader.maxwidth = 60
    pt.redraw()
    pass

def search():
    """
    Function behind the search command.

    - search_by_col: the part the user sort by.
    - text: contents we want to search
    - pt: the final result we will show on the window
    """
    global TEXT
    if TEXT!=1:
        TEXT.destroy()
        TEXT = 1
    search_by_col = comboxlist_search_col.get() ### get the current keywords the user want to search
    if search_by_col == "Trip Length":
        text = [search_text1.get(),search_text2.get()]
    elif "DirectionTime" in search_by_col:
        text = [search_text3.get(),search_text4.get(),
               search_text5.get(),search_text6.get(),
               search_text7.get(),search_text8.get(),]
    else:
        text = search_text.get() ### get the contents users want to search
    if search_by_col =="----choose type":
        mbox.showerror('Python Error Message','Error: You show first choose the key word!')  
        return
        
    search_ = my_search(df, search_by_col, text)
    global df_now
    frame = Frame(window,height = 700, width = 1200)
    frame.pack(fill='both', expand=True)
    frame.place(x=5,y=150)
    df_now = search_
    pt = Table(frame, dataframe=search_,height = 700,width = 1200)
    pt.show()
    pt.rowheader.maxwidth = 60
    pt.redraw()
    pass

def Reset():
    """
    Function behind the reset command.
    show the original data
    
    """
    global df_now
    global TEXT
    if TEXT!=1:
        TEXT.destroy()
        TEXT = 1
    frame = Frame(window,height = 700, width = 1300)
    frame.pack(fill='both', expand=True)
    frame.place(x=5,y=150)
    df_now = df
    pt = Table(frame, dataframe=df,height = 700,width = 1200)
    pt.show()
    pt.rowheader.maxwidth = 60
    pt.redraw()
    pass
def gohome():
    """
    Function behind the home command.
    show the description of our system
    
    """
    global TEXT
    TEXT = Text(window,height = 700,width=1200,font=("microsoft yahei", 20))
    TEXT.insert(END,"\n\n\n")
    TEXT.insert(END,"\t\tWelcome to our inquiry system for Taiwan traffic data.\n")
    TEXT.insert(END,"\t\tIn the system, you can\n")
    TEXT.insert(END,"\t\t1.Search various types of data\n")
    TEXT.insert(END,"\t\t2.Sort the data in any way you like\n\n")
    TEXT.insert(END,"\t\tDescription for each type:\n")
    for i in range(len(description)):
        TEXT.insert(END,'\t\t'+description[i])
        TEXT.insert(END,"\n")
    TEXT.configure(state='disabled')
    TEXT.pack()
    TEXT.place(x=5,y=150)

def callback(event):
    typ = comvalue_search_col.get()
    global search_text,lable_contents,lable_contents1,lable_contents2,search_text1,search_text2
    global lable_contents3,lable_contents4,search_text3,search_text4,search_text5,search_text6,search_text7,search_text8
    if typ == "Trip Length":
        if lable_contents:
            search_text.destroy()
            lable_contents.destroy()
            search_text = 0
            lable_contents = 0
        if lable_contents3:
            lable_contents3.destroy()
            lable_contents4.destroy()
            search_text3.destroy()
            search_text4.destroy()
            search_text5.destroy()
            search_text6.destroy()
            search_text7.destroy()
            search_text8.destroy()
            lable_contents3=0
            lable_contents4=0
            search_text3=0
            search_text4=0
            search_text5=0
            search_text6=0
            search_text7=0
            search_text8=0
        if not lable_contents1:
            lable_contents1=Label(window,text="Between",font=("Arial", 15))
            lable_contents1.place(x=550,y=85)
            lable_contents2=Label(window,text="and",font=("Arial", 15))
            lable_contents2.place(x=550+175,y=85)
            search_text1 = Entry(window,width = 5,font=("Arial", 12))
            search_text1.place(x=650,y=90)
            search_text2 = Entry(window,width = 5,font=("Arial", 12))
            search_text2.place(x=650+125,y=90)
    elif "DirectionTime_D" == typ or "DirectionTime_O" == typ:
        if lable_contents1:
            search_text1.destroy()
            search_text2.destroy()
            lable_contents1.destroy()
            lable_contents2.destroy()
            lable_contents1=0
            lable_contents2=0
            search_text1=0
            search_text2=0
        if lable_contents:
            search_text.destroy()
            lable_contents.destroy()
            search_text = 0
            lable_contents = 0
        if not lable_contents3:
            lable_contents3=Label(window,text="From 2019/08/30",font=("Arial", 15))
            lable_contents3.place(x=550,y=70)
            lable_contents4=Label(window,text="To     2019/08/30",font=("Arial", 15))
            lable_contents4.place(x=550,y=100)
            search_text3 = entry(window,placeholder = '(hh)',width = 4,font=("Arial", 12))
            search_text3.place(x=720,y=75)
            search_text4 = entry(window,placeholder = '(mm)',width = 4,font=("Arial", 12))
            search_text4.place(x=770,y=75)
            search_text5 = entry(window,placeholder = '(ss)',width = 4,font=("Arial", 12))
            search_text5.place(x=820,y=75)
            search_text6 = entry(window,placeholder = '(hh)',width = 4,font=("Arial", 12))
            search_text6.place(x=720,y=105)
            search_text7 = entry(window,placeholder = '(mm)',width = 4,font=("Arial", 12))
            search_text7.place(x=770,y=105)
            search_text8 = entry(window,placeholder = '(ss)',width = 4,font=("Arial", 12))
            search_text8.place(x=820,y=105)
    else: 
        if lable_contents1!=0:
            lable_contents1.destroy()
            lable_contents2.destroy()
            search_text1.destroy()
            search_text2.destroy()
            lable_contents1=0
            lable_contents2=0
            search_text1=0
            search_text2=0
        if lable_contents3!=0:
            lable_contents3.destroy()
            lable_contents4.destroy()
            search_text3.destroy()
            search_text4.destroy()
            search_text5.destroy()
            search_text6.destroy()
            search_text7.destroy()
            search_text8.destroy()
            lable_contents3=0
            lable_contents4=0
            search_text3=0
            search_text4=0
            search_text5=0
            search_text6=0
            search_text7=0
            search_text8=0
        if not lable_contents:
            lable_contents = Label(window,text="Contents",font=("Arial", 15))
            lable_contents.place(x=550,y=85)
            search_text = Entry(window,width = 22,font=("Arial", 12))
            search_text.place(x=650,y=90)
#         search_text4 = Entry(window,width = 5,font=("Arial", 12))
#         search_text4.place(x=650+125,y=100)

# initial the interface
TEXT = Text(window,height = 700,width=1200,font=("Courier", 20, "italic"))
TEXT.insert(END,"\n\n\n")
TEXT.insert(END,"\t\tWelcome to our inquiry system for Taiwan traffic data.\n")
TEXT.insert(END,"\t\tIn the system, you can\n")
TEXT.insert(END,"\t\t1.Search various types of data\n")
TEXT.insert(END,"\t\t2.Sort the data in any way you like\n\n")
TEXT.insert(END,"\t\tDescription for each type:\n")
for i in range(len(description)):
    TEXT.insert(END,'\t\t'+description[i])
    TEXT.insert(END,"\n")
TEXT.configure(state='disabled')
TEXT.pack()
TEXT.place(x=5,y=150)

### define the labels for sorting part
Label(window,text="Sort:",font=("Courier", 20, "bold", "italic")).place(x=20,y=20)
Label(window,text='Key words',font=("Arial", 15)).place(x=150,y=25)
Label(window,text="Methods",font=("Arial", 15)).place(x=550,y=25)

###define the labels for searching part
Label(window,text="Search:",font=("Courier", 20, "bold", "italic")).place(x=20,y=80)
Label(window,text='Key words',font=("Arial", 15)).place(x=150,y=85)
lable_contents = Label(window,text="Contents",font=("Arial", 15))
lable_contents.place(x=550,y=85)

# define the different attributes of the data we want to get
comvalue1 = StringVar()
comvalue2 = StringVar()
comboxlist1=ttk.Combobox(window,textvariable=comvalue1,font=("Arial", 12),width = 20,height=20)
comboxlist2=ttk.Combobox(window,textvariable=comvalue2,font=("Arial", 12),width = 20,height=20)
comboxlist1["values"]=("----choose type","Vehicle Type","DirectionTime_O","GantryID_O","DirectionTime_D",
                      "GantryID_D","Trip Length","Trip End","Trip Information")
comboxlist2["values"]=('----choose method','Ascending', 'Descending')
comboxlist1.current(0)
comboxlist2.current(0)
comboxlist1.pack()
comboxlist2.pack()
comboxlist2.place(x=650,y=30)
comboxlist1.place(x=300,y=30)

# define attribute we set to search
lable_contents1=0
lable_contents2=0
search_text1=0
search_text2=0
lable_contents3=0
lable_contents4=0
search_text3=0
search_text4=0
search_text5=0
search_text6=0
search_text7=0
search_text8=0
search_text = Entry(window,width = 22,font=("Arial", 12))
search_text.place(x=650,y=90)
comvalue_search_col = StringVar()
comboxlist_search_col=ttk.Combobox(window,textvariable=comvalue_search_col,font=("Arial", 12),width = 20,height=20)
comboxlist_search_col["values"]=("----choose type","Vehicle Type","DirectionTime_O","GantryID_O","DirectionTime_D",
                      "GantryID_D","Trip Length","Trip End","Trip Information")
comboxlist_search_col.current(0)

comboxlist_search_col.bind('<<ComboboxSelected>>', callback)
comboxlist_search_col.pack()
comboxlist_search_col.place(x=300,y=90)



# define the "start" button
button_sort = Button(window,text="Sort",font=("Arial", 15),command=sort,width=15,height = 1).place(x=900,y=20)
button_search = Button(window,text="Search",font=("Arial", 15),command=search,width=15,height = 1).place(x=900,y=80)
button_homepage = Button(window,text="Homepage",font=("Arial", 20),command=gohome,width=8,height = 1).place(x=1120,y=12)
button_reset = Button(window,text="Clear output",font=("Arial", 20),command=Reset,width=8,height = 1).place(x=1120,y=72)
window.mainloop()